- Usaremos diversas bibliotecas: nltk, pandas e scikit-learn
- Lembre se que algumas bibliotecas devem ser instaladas!
- Usei "pip install *nome da biblioteca*" pra fazer a instalação

In [1]:
import nltk
import re
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [ ]:
# Ler o arquivo com os dados, mostre uma amostra do arquivo e exibe a
# contagem de cada uma das colunas.

In [4]:
#dataset = pd.read_csv('/Users/rodrigomiani/Dropbox/Trabalho/UFU/Disciplinas/Organização e Recuperação da Informação/Exercícios/TP/TP4/Tweets_Mg.csv',encoding='utf-8')
dataset = pd.read_csv('Tweets_Mg.csv', encoding="utf-8")

In [5]:
dataset.head()

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
dataset.count()

Unnamed: 0                   8199
Created At                   8199
Text                         8199
Geo Coordinates.latitude      104
Geo Coordinates.longitude     104
User Location                5489
Username                     8199
User Screen Name             8199
Retweet Count                8199
Classificacao                8199
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [ ]:
# Vamos contar quantos tweets de cada tipo existem: neutro, positivo e negativo.

In [7]:
dataset[dataset.Classificacao=='Neutro'].count()

Unnamed: 0                   2453
Created At                   2453
Text                         2453
Geo Coordinates.latitude      102
Geo Coordinates.longitude     102
User Location                1712
Username                     2453
User Screen Name             2453
Retweet Count                2453
Classificacao                2453
Observação                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [8]:
dataset[dataset.Classificacao=='Positivo'].count()

Unnamed: 0                   3300
Created At                   3300
Text                         3300
Geo Coordinates.latitude        1
Geo Coordinates.longitude       1
User Location                2118
Username                     3300
User Screen Name             3300
Retweet Count                3300
Classificacao                3300
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [9]:
dataset[dataset.Classificacao=='Negativo'].count()

Unnamed: 0                   2446
Created At                   2446
Text                         2446
Geo Coordinates.latitude        1
Geo Coordinates.longitude       1
User Location                1659
Username                     2446
User Screen Name             2446
Retweet Count                2446
Classificacao                2446
Observação                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [ ]:
# Precisamos criar váriaveis diferentes para armazenar os tweets e a sua classificação

In [10]:
tweets = dataset['Text'].values
classes = dataset['Classificacao'].values

In [ ]:
# Vamos treinar o nosso primeiro modelo de classificação de texto.
# Algumas coisas são importantes aqui:

# 1) Precisamos definir como representar os tweets (BoW)
# 2) Precisamos definir qual algoritmo de classificação será usado


### Breve explicação sobre o funcionamento de algoritmos de classificação: ###
# O algoritmo irá relacionar o conteúdo da BoW (1,0,2,0,1) com a respectiva classificação 'Neutro'. Usando diferentes critérios e com auxílio dos dados
# analisados, o algoritmo irá criar "regras" para identificar/generalizar cada uma das classificações - Neutro, Positivo e Negativo. Chamamos esse conjunto de regras de "modelo".
# Feito isso, quando o modelo receber um novo tweet (BoW) sem a classificação, com base nas regras que foram criadas, ele irá tentar "adivinhar" qual será a classe daquele tweet.
# ####

In [ ]:
# Para resolver o problema 1), vamos usar somente o TF - CountVectorizer. Essa função já limpa aqueles caracteres esquisitos que vimos lá em cima.

# Na linha 1, criamos um objeto do tipo CountVectorizer chamado vectorizer. Após isso, na linha 2, usamos o objeto vectorizer para calcular a frequência de todas as palavras da lista de tweets e armazenamos seu retorno em freq_tweets.

In [11]:
vectorizer = TfidfVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(tweets)

In [ ]:
# Para resolver o problema 2), vamos trabalhar com um algoritmo de classificação chamado de Naive Bayes. Ele é baseado em probabilidades.
# Na linha 1, criamos um objeto chamado modelo do tipo Naive Bayes Multinomial.
# Na linha 2, treinamos o modelo usando a frequência de palavras (freq_tweets) e as classes de cada instância.

In [12]:
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

MultinomialNB()

In [ ]:
# Vamos fazer alguns testes "manuais". Ou seja, fornecer como entrada para o modelo alguns tweets
# e deixar que ele faça a classificação. Na opinião de vocês, qual seria a classificação para cada um desses
# tweets?

In [13]:
testes = ['O governo de Minas é uma tragédia, muito ruim','Estou muito feliz com o governo de Minas esse ano','O estado de Minas Gerais decretou calamidade financeira!!!','A segurança do estado está deixando a desejar','O governador de Minas é do Novo']
print(testes)

['O governo de Minas é uma tragédia, muito ruim', 'Estou muito feliz com o governo de Minas esse ano', 'O estado de Minas Gerais decretou calamidade financeira!!!', 'A segurança do estado está deixando a desejar', 'O governador de Minas é do Novo']


In [ ]:
# Calculo a BoW dos tweets dentro da variável testes usando o TF.

In [14]:
freq_testes = vectorizer.transform(testes)

In [ ]:
# Faço a classificação dos tweets de testes usando o modelo treinado.

In [ ]:
modelo.predict(freq_testes)

array(['Negativo', 'Neutro', 'Negativo', 'Neutro', 'Positivo'],
      dtype='<U8')

In [ ]:
# Vimos que o modelo funciona! Tem alguns erros mas isso faz parte do processo.
# O próximo passo agora é fazer uma avaliação mais robusta do modelo.
# Vamos usar uma parte da base de dados para treinar e a outra parte para testar.
# Uma maneira de se fazer isso é usando um método chamado de "Cross Validation" ou "Validação cruzada".
# Essta técnica consiste em dividir todo o conjunto de dados em K partes, que serão chamadas de folds.
# Dessas partes, uma será separada para teste e as outras restantes serão usadas para treinar o modelo.

## Exemplo ##

# Para k = 10 , imagine que todo nosso dado de treino foi dividido em 10 partes distintas.
# Assim, o modelo será treinado com 9 partes, e testado com a parte restante. Esse processo é repetido até que o modelo seja treinado e testado com todas as partes do dado.

# A variável "resultados" guarda as previsões feitas pelo pelo modelo usando a validação cruzada.

In [15]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [ ]:
# Pronto! Modelo treinado e validado! Como descobrir o desempenho do modelo? Inicialmente, usaremos uma
# medida chamada de Acurácia que nada mais é do que o percentual de acertos que o modelo teve.

In [16]:
metrics.accuracy_score(classes,resultados)

0.8768142456397121

In [ ]:
# E se eu quiser saber o desempenho por cada uma das classes? Talvez o modelo acerte mais uma classe
# do que a outra...

In [17]:
print(metrics.classification_report(classes,resultados))

              precision    recall  f1-score   support

    Negativo       0.88      0.93      0.90      2446
      Neutro       0.83      0.79      0.81      2453
    Positivo       0.91      0.90      0.91      3300

    accuracy                           0.88      8199
   macro avg       0.87      0.87      0.87      8199
weighted avg       0.88      0.88      0.88      8199



In [ ]:
# E seu eu quiser saber a quantidade de acertos por classe? Nesse caso precisamos mostrar
# a matriz de confusão.

In [18]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   Negativo  Neutro  Positivo   All
Real                                      
Negativo      2281     143        22  2446
Neutro         257    1933       263  2453
Positivo        61     264      2975  3300
All           2599    2340      3260  8199


In [ ]:
# E aí? Terminou? Satisfeito com os resultados? Será que não dá pra melhorar?
# Um primeiro ponto seria alterar a BoW. Uma outra forma de modelar isso é usando o conceito de n-grams.
## Exemplo ##
# Na frase: “Eu não gosto desse governo”, na modelagem inicial, passamos para o modelo cada palavra sendo uma feature, ficaria
# a) {eu, não, gosto, desse, governo}
# Usando Bigrams, passaríamos para o modelo 2 palavras, veja:
# b) {eu não, não gosto, gosto desse, desse governo}

## Ou seja, estamos dizendo que uma palavra tem alguma relação com outra palavra que vem logo a seguir. Lembra a história da independência de termos?

## Para implementar o bigrama basta usar o parâmetro ngram_range a seguir.
## A documentação da função CountVectorizer diz o seguinte:
# For example an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams.

In [19]:
vectorizer = CountVectorizer(ngram_range=(1,2))
freq_tweets = vectorizer.fit_transform(tweets)
modelo = LogisticRegression(multi_class='multinomial', max_iter=1000)
modelo.fit(freq_tweets,classes)
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [20]:
metrics.accuracy_score(classes,resultados)

0.8947432613733383

In [21]:
print(metrics.classification_report(classes,resultados))

              precision    recall  f1-score   support

    Negativo       0.93      0.88      0.91      2446
      Neutro       0.79      0.91      0.84      2453
    Positivo       0.97      0.90      0.93      3300

    accuracy                           0.89      8199
   macro avg       0.89      0.89      0.89      8199
weighted avg       0.90      0.89      0.90      8199



In [22]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   Negativo  Neutro  Positivo   All
Real                                      
Negativo      2160     276        10  2446
Neutro         141    2221        91  2453
Positivo        18     327      2955  3300
All           2319    2824      3056  8199
